In [1]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

25/03/05 17:02:09 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.36.156.69 instead (on interface en0)
25/03/05 17:02:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 17:02:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Build RDD

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime 
start = datetime(year = 2020, month = 1, day = 1)

# Filter
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

# Map
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute = 0, second = 0, microsecond = 0)
    zone = row.PULocationID
    
    amount = row.total_amount
    count = 1 
    
    # Establish key-value pair for groupBy later
    key = (hour, zone)
    value = (amount, count)
    
    return (key, value)

### Reduce
We are going to *reduce by key*, generating a `(key, reduced_value)` pair.

In [5]:
def calculate_revenue(left_value, right_value): 
    
    # Unpack tuple
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    output_value = (output_amount, output_count)
    
    return output_value

### Unwrap
Unwrapping the aggregation

In [6]:
from collections import namedtuple

RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [7]:
def unwrap(row): 
    return RevenueRow(
        hour = row[0][0], 
        zone = row[0][1], 
        revenue = row[1][0], 
        count = row[1][1]
    )

# Without namedtuple, the resultant DataFrame wouldn't have column names, 
# as RDDs inherently don't use them!
# We will also generate a more specific schema and feed that to `.toDF()`.

In [8]:
from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [9]:

df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [10]:
# df_result.write.parquet('tmp/green-revenue')

This effectively reproduces what we did in `06_spark_sql.ipynb`!

## `mapPartitions()`

In [11]:
cols = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(cols) \
    .rdd



In [12]:
import pandas as pd

In [19]:
# model = ...

def model_predict(df): 
    
#     # FAKE model predictions...
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    
    return y_pred


In [20]:


def apply_model_in_batch(rows):
    
    df = pd.DataFrame(rows, columns = cols)
    predictions = model_predict(df)
    
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        
        yield row 

In [21]:
# `.collect()` collects all elements of the RDD and returns a list

df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [22]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|              15.1|
|6.6000000000000005|
|              4.45|
|             14.15|
|              6.65|
|             17.05|
|               7.5|
|               1.0|
|12.350000000000001|
|              97.6|
|               2.6|
|              40.9|
|             14.75|
|               7.9|
|               4.3|
|              79.2|
| 6.550000000000001|
|3.5999999999999996|
|               5.5|
+------------------+
only showing top 20 rows

